### Importar bibliotecas

In [12]:
# Conexão com BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account
# Manipulação de dados
import pandas as pd
import numpy as np
import warnings

### Criar conexão com os dados no GCP

In [13]:
# Realizar conexão com o GCP
credentials = service_account.Credentials.from_service_account_file("../keys/ml-na-saude-ed1fc3c1a83e.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

### Criar dataframe com os dados já tratados e selecionados

In [14]:
# Os dados de mortalidade já foram tratados na ETL para calcular o YLL por cada óbito
# Aqui esta importação já está trazendo a taxa média do YLL dos municípios de médio porte por quedrimestre
consulta_sql = """
select
  a.quadrimestre,
  avg(a.taxa_yll) as taxa_media_yll
from (
with yll_quadrimestral as (
	select
		case
			when extract(month from y.dt_obito) between 1 and 4 then date(extract(year from y.dt_obito), 4, 30)
			when extract(month from y.dt_obito) between 5 and 8 then date(extract(year from y.dt_obito), 8, 31)
			when extract(month from y.dt_obito) between 9 and 12 then date(extract(year from y.dt_obito), 12, 31)
			end as quadrimestre,
		m.nm_municipio,
		p.populacao,
		sum(y.yll) as soma_yll
	from `ml-na-saude.yll_por_obito.yll` y
	join `ml-na-saude.yll_por_obito.populacao` p on y.cd_mun_res = p.cd_municipio and y.ano_obito = p.ano
	join `ml-na-saude.yll_por_obito.municipio` m on p.cd_municipio = m.cd_municipio
	where p.porte = 'Médio Porte'
    group by 1,2,3
)
select
	quadrimestre,
	nm_municipio,
  soma_yll,
  populacao,
  soma_yll / populacao * 1000 as taxa_yll
from yll_quadrimestral
group by 1,2,3,4
) a
group by 1
order by 1
"""

In [15]:
# Ignorar avisos e gerar dataframe
warnings.simplefilter("ignore")
df = client.query(consulta_sql).to_dataframe()

In [16]:
df.head()

,quadrimestre,taxa_media_yll
0,2010-04-30,6.427726
1,2010-08-31,6.767661
2,2010-12-31,6.111099
3,2011-04-30,6.550037
4,2011-08-31,6.878076


In [17]:
df.dtypes

quadrimestre       dbdate
taxa_media_yll    float64
dtype: object

### Cópia do DataFrame para manipulação e criação dos modelos

In [31]:
# Copiar dataframe para manipular dados
time_series = df.copy()

In [32]:
# Transformando o quadrimestre em data
time_series['quadrimestre'] = pd.to_datetime(time_series['quadrimestre'])

In [33]:
time_series.dtypes

quadrimestre      datetime64[ns]
taxa_media_yll           float64
dtype: object

In [34]:
time_series

,quadrimestre,taxa_media_yll
0,2010-04-30,6.427726
1,2010-08-31,6.767661
2,2010-12-31,6.111099
3,2011-04-30,6.550037
4,2011-08-31,6.878076
5,2011-12-31,5.804813
6,2012-04-30,6.280719
7,2012-08-31,6.739170
8,2012-12-31,6.049970
9,2013-04-30,6.253954


### Modelo